In [ ]:
import tensorflow as tf
import numpy as np
from functools import partial

xy = np.loadtxt("data2.csv", delimiter=",", dtype=np.float32)
x_data = xy[0:-178, 0:-1]
y_data = xy[0:-178, [-1]]
x_validation_data = xy[713:, 0:-1]
y_validation_data = xy[713:, [-1]]
n_inputs = 7
n_hidden1 = 300
n_hidden2 = 300
n_hidden3 = 300
n_hidden4 = 300
n_hidden5 = 300
n_hidden6 = 300
n_hidden7 = 300
n_hidden8 = 300
n_hidden9 = 300
n_outputs = 1
learning_rate = 0.001
momentum = 0.9
n_epochs = 10001
scale = 0.001

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
Y = tf.placeholder(tf.float32, shape=(None,1), name="Y")

my_dense_layer = partial(tf.layers.dense, activation=tf.nn.relu, kernel_regularizer=tf.contrib.layers.l1_regularizer(scale))


with tf.name_scope("dnn"):
    hidden1 = my_dense_layer(X, n_hidden1, name = "hidden1")
    hidden2 = my_dense_layer(hidden1, n_hidden2, name = "hidden2")
    hidden3 = my_dense_layer(hidden2, n_hidden3, name = "hidden3")
    hidden4 = my_dense_layer(hidden3, n_hidden4, name = "hidden4")
    hidden5 = my_dense_layer(hidden4, n_hidden5, name = "hidden5")
    hidden6 = my_dense_layer(hidden5, n_hidden6, name = "hidden6")
    hidden7 = my_dense_layer(hidden6, n_hidden7, name = "hidden7")
    hidden8 = my_dense_layer(hidden7, n_hidden8, name = "hidden8")
    hidden9 = my_dense_layer(hidden8, n_hidden9, name = "hidden9")
    logits = my_dense_layer(hidden9, n_outputs, activation = None, name = "outputs")
     
with tf.name_scope("loss"):
    xentropy = tf.nn.sigmoid_cross_entropy_with_logits(labels = Y, logits = logits)
    
    base_loss = tf.reduce_mean(xentropy, name="avg_xentropy")
    reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    loss = tf.add_n([base_loss] + reg_losses, name = "loss")
    
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    predicted = tf.nn.sigmoid(logits)
    correct = tf.equal(tf.round(predicted), Y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))


init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict={X: x_data, Y: y_data})
        acc_train = accuracy.eval(feed_dict={X: x_data, Y: y_data})
        acc_val = accuracy.eval(feed_dict={X: x_validation_data, Y: y_validation_data})
        if epoch % 200 == 0:
            print(epoch, " Train accuracy: ", acc_train, " Validation accuracy: ", acc_val)